In [ ]:
pip install git+https://github.com/billybraith17/billiespnet

In [ ]:
pip install torch espnet espnet_model_zoo parallel_wavegan==0.5.4

In [ ]:
%%bash
!(stat -t /usr/local/lib/*/dist-packages/google/colab > /dev/null 2>&1) && exit
pip install ninja 2>> install.log
git clone https://github.com/SIDN-IAP/global-model-repr.git tutorial_code 2>> install.log
# git clone https://github.com/billybraith17/global-model-repr.git tutorial_code 2>> install.log

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
try: # set up path
    import google.colab, sys, torch
    sys.path.append('/content/tutorial_code')
    if not torch.cuda.is_available():
        print("Change runtime type to include a GPU.")
except:
    pass

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib as mpl

In [ ]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Running pytorch', torch.__version__, 'using', device.type)

Running pytorch 1.12.1+cu113 using cuda


In [ ]:
import torchvision
import torch.hub

# using the global-model-repr package
from netdissect import nethook, proggan

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
# def from_pretrained2(
#     model_tag: Optional[str] = None,
#     vocoder_tag: Optional[str] = None,
#     **kwargs: Optional[Any],
# ):
"""Build Text2Speech instance from the pretrained model.
Args:
    model_tag (Optional[str]): Model tag of the pretrained models.
        Currently, the tags of espnet_model_zoo are supported.
    vocoder_tag (Optional[str]): Vocoder tag of the pretrained vocoders.
        Currently, the tags of parallel_wavegan are supported, which should
        start with the prefix "parallel_wavegan/".
Returns:
    Text2Speech: Text2Speech instance.
"""

from espnet2.bin.tts_inference import Text2Speech

model_tag = 'kan-bayashi/vctk_xvector_conformer_fastspeech2' #@param ["kan-bayashi/vctk_gst_tacotron2", "kan-bayashi/vctk_gst_transformer", "kan-bayashi/vctk_xvector_tacotron2", "kan-bayashi/vctk_xvector_transformer", "kan-bayashi/vctk_xvector_conformer_fastspeech2", "kan-bayashi/vctk_gst+xvector_tacotron2", "kan-bayashi/vctk_gst+xvector_transformer", "kan-bayashi/vctk_gst+xvector_conformer_fastspeech2", "kan-bayashi/vctk_multi_spk_vits", "kan-bayashi/vctk_full_band_multi_spk_vits", "kan-bayashi/libritts_xvector_transformer", "kan-bayashi/libritts_xvector_conformer_fastspeech2", "kan-bayashi/libritts_gst+xvector_transformer", "kan-bayashi/libritts_gst+xvector_conformer_fastspeech2", "kan-bayashi/libritts_xvector_vits"] {type:"string"}
vocoder_tag = "parallel_wavegan/vctk_hifigan.v1"  #@param ["none", "parallel_wavegan/vctk_parallel_wavegan.v1.long", "parallel_wavegan/vctk_multi_band_melgan.v2", "parallel_wavegan/vctk_style_melgan.v1", "parallel_wavegan/vctk_hifigan.v1", "parallel_wavegan/libritts_parallel_wavegan.v1.long", "parallel_wavegan/libritts_multi_band_melgan.v2", "parallel_wavegan/libritts_hifigan.v1", "parallel_wavegan/libritts_style_melgan.v1"] {type:"string"}
kwargs = {
    # Only for Tacotron 2 & Transformer
    "threshold" : 0.5,
    # Only for Tacotron 2
    "minlenratio" : 0.0,
    "maxlenratio" : 10.0,
    "use_att_constraint" : False,
    "backward_window" : 1,
    "forward_window" : 3,
    # Only for FastSpeech & FastSpeech2 & VITS
    "speed_control_alpha" : 1.0,
    # Only for VITS
    "noise_scale" : 0.333,
    "noise_scale_dur" : 0.333,
}

# print(kwargs["noise_scale"])
# model_tag: Optional[str] = None
# vocoder_tag: Optional[str] = None
# **kwargs: Optional[Any]


if model_tag is not None:
    try:
        from espnet_model_zoo.downloader import ModelDownloader

    except ImportError:
        logging.error(
            "`espnet_model_zoo` is not installed. "
            "Please install via `pip install -U espnet_model_zoo`."
        )
        raise
    d = ModelDownloader()
    kwargs.update(**d.download_and_unpack(model_tag))

if vocoder_tag is not None:
    if vocoder_tag.startswith("parallel_wavegan/"):
        try:
            from parallel_wavegan.utils import download_pretrained_model

        except ImportError:
            logging.error(
                "`parallel_wavegan` is not installed. "
                "Please install via `pip install -U parallel_wavegan`."
            )
            raise

        from parallel_wavegan import __version__
        from packaging.version import parse as V
        from pathlib import Path

        # NOTE(kan-bayashi): Filelock download is supported from 0.5.2
        assert V(__version__) > V("0.5.1"), (
            "Please install the latest parallel_wavegan "
            "via `pip install -U parallel_wavegan`."
        )
        vocoder_tag = vocoder_tag.replace("parallel_wavegan/", "")
        vocoder_file = download_pretrained_model(vocoder_tag)
        vocoder_config = Path(vocoder_file).parent / "config.yml"
        kwargs.update(vocoder_config=vocoder_config, vocoder_file=vocoder_file)

    else:
        raise ValueError(f"{vocoder_tag} is unsupported format.")

# __init__
text2speech = Text2Speech(**kwargs)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package cmudict to /root/nltk_data...
[nltk_data]   Unzipping corpora/cmudict.zip.
https://zenodo.org/record/4394602/files/tts_train_xvector_conformer_fastspeech2_transformer_teacher_raw_phn_tacotron_g2p_en_no_space_train.loss.ave.zip?download=1: 100%|██████████| 270M/270M [00:21<00:00, 13.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1oVOC4Vf0DYLdDp4r7GChfgj7Xh5xd0ex
To: /root/.cache/parallel_wavegan/vctk_hifigan.v1.tar.gz
100%|██████████| 916M/916M [00:03<00:00, 276MB/s]


In [ ]:
# Wrap the vocoder
from netdissect import nethook
# layer = 'vocoder.blocks.11.convs2.0.1'    #@param ["vocoder.last_conv_layers.0", "vocoder.last_conv_layers.1", "vocoder.last_conv_layers.2", "vocoder.last_conv_layers.3", "vocoder.conv_layers.6.conv", "vocoder.conv_layers.6.conv1x1_aux", "vocoder.conv_layers.6.conv1x1_out", "vocoder.conv_layers.6.conv1x1_skip", "vocoder.first_conv", "vocoder.upsample_net.conv_in", "vocoder.upsample_net.upsample.up_layers.6", "vocoder.upsample_net.upsample.up_layers.7", "vocoder.melgan.0", "vocoder.melgan.19.stack.4", "vocoder.melgan.23", "vocoder.pqmf.pad_fn", "vocoder.blocks.7.upsample", "vocoder.output_conv.1", "vocoder.blocks.11.convs2.0.1", "vocoder.output_conv.2"] {type:"string"}

# layer = 'vocoder.blocks.0.convs1.0.1'

# Don't re-wrap it, if it's already wrapped (e.g., if you press enter twice)
if not isinstance(text2speech.vocoder, nethook.InstrumentedModel):
    text2speech.vocoder = nethook.InstrumentedModel(text2speech.vocoder)


# convolutional layers
# HiFiGAN_layers = ['vocoder.input_conv', 'vocoder.blocks.0.convs1.0.1', 'vocoder.blocks.0.convs1.1.1', 'vocoder.blocks.0.convs1.2.1', 'vocoder.blocks.0.convs2.0.1', 'vocoder.blocks.0.convs2.1.1', 'vocoder.blocks.0.convs2.2.1', 'vocoder.blocks.1.convs1.0.1', 'vocoder.blocks.1.convs1.1.1', 'vocoder.blocks.1.convs1.2.1', 'vocoder.blocks.1.convs2.0.1', 'vocoder.blocks.1.convs2.1.1', 'vocoder.blocks.1.convs2.2.1', 'vocoder.blocks.2.convs1.0.1', 'vocoder.blocks.2.convs1.1.1', 'vocoder.blocks.2.convs1.2.1', 'vocoder.blocks.2.convs2.0.1', 'vocoder.blocks.2.convs2.1.1', 'vocoder.blocks.2.convs2.2.1', 'vocoder.blocks.3.convs1.0.1', 'vocoder.blocks.3.convs1.1.1', 'vocoder.blocks.3.convs1.2.1', 'vocoder.blocks.3.convs2.0.1', 'vocoder.blocks.3.convs2.1.1', 'vocoder.blocks.3.convs2.2.1', 'vocoder.blocks.4.convs1.0.1', 'vocoder.blocks.4.convs1.1.1', 'vocoder.blocks.4.convs1.2.1', 'vocoder.blocks.4.convs2.0.1', 'vocoder.blocks.4.convs2.1.1', 'vocoder.blocks.4.convs2.2.1', 'vocoder.blocks.5.convs1.0.1', 'vocoder.blocks.5.convs1.1.1', 'vocoder.blocks.5.convs1.2.1', 'vocoder.blocks.5.convs2.0.1', 'vocoder.blocks.5.convs2.1.1', 'vocoder.blocks.5.convs2.2.1', 'vocoder.blocks.6.convs1.0.1', 'vocoder.blocks.6.convs1.1.1', 'vocoder.blocks.6.convs1.2.1', 'vocoder.blocks.6.convs2.0.1', 'vocoder.blocks.6.convs2.1.1', 'vocoder.blocks.6.convs2.2.1', 'vocoder.blocks.7.convs1.0.1', 'vocoder.blocks.7.convs1.1.1', 'vocoder.blocks.7.convs1.2.1', 'vocoder.blocks.7.convs2.0.1', 'vocoder.blocks.7.convs2.1.1', 'vocoder.blocks.7.convs2.2.1', 'vocoder.blocks.8.convs1.0.1', 'vocoder.blocks.8.convs1.1.1', 'vocoder.blocks.8.convs1.2.1', 'vocoder.blocks.8.convs2.0.1', 'vocoder.blocks.8.convs2.1.1', 'vocoder.blocks.8.convs2.2.1', 'vocoder.blocks.9.convs1.0.1', 'vocoder.blocks.9.convs1.1.1', 'vocoder.blocks.9.convs1.2.1', 'vocoder.blocks.9.convs2.0.1', 'vocoder.blocks.9.convs2.1.1', 'vocoder.blocks.9.convs2.2.1', 'vocoder.blocks.10.convs1.0.1', 'vocoder.blocks.10.convs1.1.1', 'vocoder.blocks.10.convs1.2.1', 'vocoder.blocks.10.convs2.0.1', 'vocoder.blocks.10.convs2.1.1', 'vocoder.blocks.10.convs2.2.1', 'vocoder.blocks.11.convs1.0.1', 'vocoder.blocks.11.convs1.1.1', 'vocoder.blocks.11.convs1.2.1', 'vocoder.blocks.11.convs2.0.1', 'vocoder.blocks.11.convs2.1.1', 'vocoder.blocks.11.convs2.2.1', 'vocoder.output_conv.1']
# activation layers for convolutional layers
HiFiGAN_layers = ['vocoder.upsamples.0.0', 'vocoder.upsamples.1.0', 'vocoder.upsamples.2.0', 'vocoder.upsamples.3.0', 'vocoder.blocks.0.convs1.0.0', 'vocoder.blocks.0.convs1.1.0', 'vocoder.blocks.0.convs1.2.0', 'vocoder.blocks.0.convs2.0.0', 'vocoder.blocks.0.convs2.1.0', 'vocoder.blocks.0.convs2.2.0', 'vocoder.blocks.1.convs1.0.0', 'vocoder.blocks.1.convs1.1.0', 'vocoder.blocks.1.convs1.2.0', 'vocoder.blocks.1.convs2.0.0', 'vocoder.blocks.1.convs2.1.0', 'vocoder.blocks.1.convs2.2.0', 'vocoder.blocks.2.convs1.0.0', 'vocoder.blocks.2.convs1.1.0', 'vocoder.blocks.2.convs1.2.0', 'vocoder.blocks.2.convs2.0.0', 'vocoder.blocks.2.convs2.1.0', 'vocoder.blocks.2.convs2.2.0', 'vocoder.blocks.3.convs1.0.0', 'vocoder.blocks.3.convs1.1.0', 'vocoder.blocks.3.convs1.2.0', 'vocoder.blocks.3.convs2.0.0', 'vocoder.blocks.3.convs2.1.0', 'vocoder.blocks.3.convs2.2.0', 'vocoder.blocks.4.convs1.0.0', 'vocoder.blocks.4.convs1.1.0', 'vocoder.blocks.4.convs1.2.0', 'vocoder.blocks.4.convs2.0.0', 'vocoder.blocks.4.convs2.1.0', 'vocoder.blocks.4.convs2.2.0', 'vocoder.blocks.5.convs1.0.0', 'vocoder.blocks.5.convs1.1.0', 'vocoder.blocks.5.convs1.2.0', 'vocoder.blocks.5.convs2.0.0', 'vocoder.blocks.5.convs2.1.0', 'vocoder.blocks.5.convs2.2.0', 'vocoder.blocks.6.convs1.0.0', 'vocoder.blocks.6.convs1.1.0', 'vocoder.blocks.6.convs1.2.0', 'vocoder.blocks.6.convs2.0.0', 'vocoder.blocks.6.convs2.1.0', 'vocoder.blocks.6.convs2.2.0', 'vocoder.blocks.7.convs1.0.0', 'vocoder.blocks.7.convs1.1.0', 'vocoder.blocks.7.convs1.2.0', 'vocoder.blocks.7.convs2.0.0', 'vocoder.blocks.7.convs2.1.0', 'vocoder.blocks.7.convs2.2.0', 'vocoder.blocks.8.convs1.0.0', 'vocoder.blocks.8.convs1.1.0', 'vocoder.blocks.8.convs1.2.0', 'vocoder.blocks.8.convs2.0.0', 'vocoder.blocks.8.convs2.1.0', 'vocoder.blocks.8.convs2.2.0', 'vocoder.blocks.9.convs1.0.0', 'vocoder.blocks.9.convs1.1.0', 'vocoder.blocks.9.convs1.2.0', 'vocoder.blocks.9.convs2.0.0', 'vocoder.blocks.9.convs2.1.0', 'vocoder.blocks.9.convs2.2.0', 'vocoder.blocks.10.convs1.0.0', 'vocoder.blocks.10.convs1.1.0', 'vocoder.blocks.10.convs1.2.0', 'vocoder.blocks.10.convs2.0.0', 'vocoder.blocks.10.convs2.1.0', 'vocoder.blocks.10.convs2.2.0', 'vocoder.blocks.11.convs1.0.0', 'vocoder.blocks.11.convs1.1.0', 'vocoder.blocks.11.convs1.2.0', 'vocoder.blocks.11.convs2.0.0', 'vocoder.blocks.11.convs2.1.0', 'vocoder.blocks.11.convs2.2.0', 'vocoder.output_conv.0', 'vocoder.output_conv.2']

parallel_wavegan_layers = ["vocoder.first_conv", "vocoder.conv_layers.0.conv", "vocoder.conv_layers.0.conv1x1_aux", "vocoder.conv_layers.0.conv1x1_out", "vocoder.conv_layers.0.conv1x1_skip", "vocoder.conv_layers.1.conv", "vocoder.conv_layers.1.conv1x1_aux", "vocoder.conv_layers.1.conv1x1_out", "vocoder.conv_layers.1.conv1x1_skip", "vocoder.conv_layers.2.conv", "vocoder.conv_layers.2.conv1x1_aux", "vocoder.conv_layers.2.conv1x1_out", "vocoder.conv_layers.2.conv1x1_skip", "vocoder.conv_layers.3.conv", "vocoder.conv_layers.3.conv1x1_aux", "vocoder.conv_layers.3.conv1x1_out", "vocoder.conv_layers.3.conv1x1_skip", "vocoder.conv_layers.4.conv", "vocoder.conv_layers.4.conv1x1_aux", "vocoder.conv_layers.4.conv1x1_out", "vocoder.conv_layers.4.conv1x1_skip", "vocoder.conv_layers.5.conv", "vocoder.conv_layers.5.conv1x1_aux", "vocoder.conv_layers.5.conv1x1_out", "vocoder.conv_layers.5.conv1x1_skip", "vocoder.conv_layers.6.conv", "vocoder.conv_layers.6.conv1x1_aux", "vocoder.conv_layers.6.conv1x1_out", "vocoder.conv_layers.6.conv1x1_skip", "vocoder.conv_layers.7.conv", "vocoder.conv_layers.7.conv1x1_aux", "vocoder.conv_layers.7.conv1x1_out", "vocoder.conv_layers.7.conv1x1_skip", "vocoder.conv_layers.8.conv", "vocoder.conv_layers.8.conv1x1_aux", "vocoder.conv_layers.8.conv1x1_out", "vocoder.conv_layers.8.conv1x1_skip", "vocoder.conv_layers.9.conv", "vocoder.conv_layers.9.conv1x1_aux", "vocoder.conv_layers.9.conv1x1_out", "vocoder.conv_layers.9.conv1x1_skip", "vocoder.conv_layers.10.conv", "vocoder.conv_layers.10.conv1x1_aux", "vocoder.conv_layers.10.conv1x1_out", "vocoder.conv_layers.10.conv1x1_skip", "vocoder.conv_layers.11.conv", "vocoder.conv_layers.11.conv1x1_aux", "vocoder.conv_layers.11.conv1x1_out", "vocoder.conv_layers.11.conv1x1_skip", "vocoder.conv_layers.12.conv", "vocoder.conv_layers.12.conv1x1_aux", "vocoder.conv_layers.12.conv1x1_out", "vocoder.conv_layers.12.conv1x1_skip", "vocoder.conv_layers.13.conv", "vocoder.conv_layers.13.conv1x1_aux", "vocoder.conv_layers.13.conv1x1_out", "vocoder.conv_layers.13.conv1x1_skip", "vocoder.conv_layers.14.conv", "vocoder.conv_layers.14.conv1x1_aux", "vocoder.conv_layers.14.conv1x1_out", "vocoder.conv_layers.14.conv1x1_skip", "vocoder.conv_layers.15.conv", "vocoder.conv_layers.15.conv1x1_aux", "vocoder.conv_layers.15.conv1x1_out", "vocoder.conv_layers.15.conv1x1_skip", "vocoder.conv_layers.16.conv", "vocoder.conv_layers.16.conv1x1_aux", "vocoder.conv_layers.16.conv1x1_out", "vocoder.conv_layers.16.conv1x1_skip", "vocoder.conv_layers.17.conv", "vocoder.conv_layers.17.conv1x1_aux", "vocoder.conv_layers.17.conv1x1_out", "vocoder.conv_layers.17.conv1x1_skip", "vocoder.conv_layers.18.conv", "vocoder.conv_layers.18.conv1x1_aux", "vocoder.conv_layers.18.conv1x1_out", "vocoder.conv_layers.18.conv1x1_skip", "vocoder.conv_layers.19.conv", "vocoder.conv_layers.19.conv1x1_aux", "vocoder.conv_layers.19.conv1x1_out", "vocoder.conv_layers.19.conv1x1_skip", "vocoder.conv_layers.20.conv", "vocoder.conv_layers.20.conv1x1_aux", "vocoder.conv_layers.20.conv1x1_out", "vocoder.conv_layers.20.conv1x1_skip", "vocoder.conv_layers.21.conv", "vocoder.conv_layers.21.conv1x1_aux", "vocoder.conv_layers.21.conv1x1_out", "vocoder.conv_layers.21.conv1x1_skip", "vocoder.conv_layers.22.conv", "vocoder.conv_layers.22.conv1x1_aux", "vocoder.conv_layers.22.conv1x1_out", "vocoder.conv_layers.22.conv1x1_skip", "vocoder.conv_layers.23.conv", "vocoder.conv_layers.23.conv1x1_aux", "vocoder.conv_layers.23.conv1x1_out", "vocoder.conv_layers.23.conv1x1_skip", "vocoder.conv_layers.24.conv", "vocoder.conv_layers.24.conv1x1_aux", "vocoder.conv_layers.24.conv1x1_out", "vocoder.conv_layers.24.conv1x1_skip", "vocoder.conv_layers.25.conv", "vocoder.conv_layers.25.conv1x1_aux", "vocoder.conv_layers.25.conv1x1_out", "vocoder.conv_layers.25.conv1x1_skip", "vocoder.conv_layers.26.conv", "vocoder.conv_layers.26.conv1x1_aux", "vocoder.conv_layers.26.conv1x1_out", "vocoder.conv_layers.26.conv1x1_skip", "vocoder.conv_layers.27.conv", "vocoder.conv_layers.27.conv1x1_aux", "vocoder.conv_layers.27.conv1x1_out", "vocoder.conv_layers.27.conv1x1_skip", "vocoder.conv_layers.28.conv", "vocoder.conv_layers.28.conv1x1_aux", "vocoder.conv_layers.28.conv1x1_out", "vocoder.conv_layers.28.conv1x1_skip", "vocoder.conv_layers.29.conv", "vocoder.conv_layers.29.conv1x1_aux", "vocoder.conv_layers.29.conv1x1_out", "vocoder.conv_layers.29.conv1x1_skip",  "vocoder.last_conv_layers.1", "vocoder.last_conv_layers.3"]


for layer in HiFiGAN_layers:
  text2speech.vocoder.retain_layer(layer)


# "vocoder.blocks.11.convs2.0.1", "vocoder.output_conv.2"

In [ ]:
text2speech.vocoder

# text2speech.vocoder.retain_layer('vocoder')

In [ ]:
import glob
import os
import numpy as np
import kaldiio

# Get model directory path
from espnet_model_zoo.downloader import ModelDownloader
d = ModelDownloader()
model_dir = os.path.dirname(d.download_and_unpack(model_tag)["train_config"])

# X-vector selection
# We only need the x-vector part at that's what this model uses. The ones with
spembs = None
if text2speech.use_spembs:
  xvector_ark = [p for p in glob.glob(f"{model_dir}/../../dump/**/spk_xvector.ark", recursive=True) if "tr" in p][0]
  xvectors = {k: v for k, v in kaldiio.load_ark(xvector_ark)}
  spks = list(xvectors.keys())

  # # randomly select speaker
  # random_spk_idx = np.random.randint(0, len(spks))
  # spk = spks[random_spk_idx]
  # spembs = xvectors[spk]
  # print(f"selected spk: {spk}")

In [ ]:
# acts = text2speech.vocoder.retained_layer('vocoder.blocks.0.convs1.0.0')
# acts2 = text2speech.vocoder.retained_layer('vocoder.blocks.0.convs1.1.0')
# print(acts.shape, '\n \n', acts2.shape)

# acts = text2speech.vocoder.retained_layer(layer)
# print(acts.shape)
# torch.flatten(acts, start_dim=1).shape


In [ ]:
# Indices of speakers with these properties

southern = {0, 3, 4, 6, 7, 14, 30, 31, 41}
s_african = {82, 86, 93, 99}
indian = {21, 24} # 107
scottish = {9, 11, 15, 19, 20, 22, 25, 28, 33, 35, 36, 37, 38, 44, 45, 48, 54, 57, 58}
irish = {18, 39, 56, 61, 65, 67, 81, 95, 105}
dublin = {18, 61, 65, 95}
n_irish = {12, 34, 62, 63, 73, 100}
belfast = {12, 34, 62, 63, 73}
cali = {64, 68, 69, 85}
yorkshire = {40, 43} # 60
NE = {42, 50, 55, 59}
canada = {71, 72, 76, 80, 83, 84, 97, 104}

female = {0, 3,4,5,6, 8, 9, 10, 12, 13, 14, 17, 21, 22, 23, 26, 30, 34, 35, 37, 38, 39, 40, 41, 42, 49, 50, 53, 55, 56, 61, 63, 64, 65, 66, 68, 69, 70, 72, 74, 75, 76, 77, 78, 80, 81,82, 84, 85, 86, 88, 89, 90, 92,93,94,95,96,97, 100, 102,103}
male = {1, 2, 7, 11, 15, 16, 18, 19, 20, 24, 25, 27, 28, 29, 31, 32, 33, 36, 43, 44, 45, 46, 47, 48, 51, 52, 54, 57, 58, 59, 60, 62, 67, 71, 73, 79, 83, 87, 91, 98, 99, 101, 104, 105, 106, 107}

In [ ]:
# testing to see if we could convert a list to a tensor then use topk to get the values and indices

# IoU_list = torch.tensor(IoU_list)
# top_IoU = torch.topk(IoU_list, 5)
# print(top_IoU)

In [ ]:
# # code for if I want to make speech quickly
# import soundfile as sf
# import time

# spk = spks[0]
# spembs = xvectors[spk]
# wav = text2speech(x, spembs=spembs)["wav"]

# sf.write("out.wav", wav.numpy(), 24000, "PCM_16")

# # Display the audio file below
# from IPython.display import display, Audio
# display(Audio(wav.view(-1).cpu().numpy(), rate=24000))

In [ ]:
# # testing code to generalise the proeprties indices when running more than once

# chosen_property = male
# no_of_runs = 2
# # adapt list to the number of runs
# chosen_property = np.array(list(chosen_property))
# chosen_property = chosen_property*no_of_runs
# chosen_property_big = set(chosen_property)
# for i in range(no_of_runs-1):
#   chosen_property = chosen_property+1
#   chosen_property_big = chosen_property_big.union(set(chosen_property))
# chosen_property_big

In [ ]:
# DON'T RUN
# MAKE SURE YOU CHANGE THE FILE NAME FIRST
import soundfile as sf
import time

# the ultimate loop
import pandas as pd
from netdissect import tally

# HERE
# set text we want our GAN to synthesise
text_list = ['Please call Stella. Ask her too.', 'Bring these things with her from the store.']
# x = "Please call Stella. Ask her too."

# defining speaker indices
from torch.utils.data import TensorDataset

no_of_runs = 2
zds = torch.linspace(0, 108*no_of_runs-1, 108*no_of_runs)

# dictionary to store the results
data = {}

# HERE
# property we want to calculate the IoU for
chosen_property = female
# adapt list to the number of runs
chosen_property = np.array(list(chosen_property))
chosen_property = chosen_property*no_of_runs
chosen_property_big = set(chosen_property)
for i in range(no_of_runs-1):
  chosen_property = chosen_property+1
  chosen_property_big = chosen_property_big.union(set(chosen_property))



# loop over each layer in the network
for layer in HiFiGAN_layers:
  # To collect stats, define a function that returns 2d [samples, units]
  def compute_samples(index):
      # print(index)
      # the index we will use to choose the speaker and the text we use
      spk_idx = index[0] // no_of_runs
      text_idx = index[0] % no_of_runs

      spk = spks[int(spk_idx)]
      spembs = xvectors[spk]
      wav = text2speech(text_list[int(text_idx)], spembs=spembs)["wav"]

      acts = text2speech.vocoder.retained_layer(layer)
      acts = torch.flatten(acts, start_dim=1)
      acts_stack = acts

      for i in index[1:]:
        spk_idx = i // no_of_runs
        text_idx = i % no_of_runs

        spk = spks[int(spk_idx)]
        spembs = xvectors[spk]
        wav = text2speech(text_list[int(text_idx)], spembs=spembs)["wav"]
        acts = text2speech.vocoder.retained_layer(layer)
        acts = torch.flatten(acts, start_dim=1)
        # print(acts.shape)

        # sf.write("out.wav", wav.numpy(), 24000, "PCM_16")

        # # Display the audio file below
        # from IPython.display import display, Audio
        # display(Audio(wav.view(-1).cpu().numpy(), rate=24000))

        acts_stack = torch.cat((acts_stack, acts), 0)
      return acts_stack


# use topk to collect the speaker indices which cause the units to activate most
  topk = tally.tally_topk(compute_samples, zds)

  #calculate IoUs for this layer
  IoU_list = []
  result = topk.result()[1]
  result = result.tolist()
  # len(result)

# HERE
  for i in range(0, topk.result()[1].shape[0]):
    # which quantile we want for the threshold
    top = result[i][0:10]
    top = set(top)
    num = top.intersection(chosen_property_big)
    den = top.union(chosen_property_big)
    IoU_list.append(len(num) / len(den))

# select the 20 largest IoUs in the layer and find their indices
  IoU_list = torch.tensor(IoU_list)
  top_IoU = torch.topk(IoU_list, 20)
  print(layer, '\n', top_IoU, '\n')
  data[layer] = [IoU_list, top_IoU[0], top_IoU[1]]

# HERE
# write dictionary to pandas dataframe
  frame = pd.DataFrame(data, index = ['All IoUs', 'Top IoUs', 'Indices of top IoUs'])
  frame.to_csv('female_100_10x2.csv')


  0%|          | 0/22 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:43: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
/usr/local/lib/python3.7/dist-packages/espnet2/torch_utils/device_funcs.py:29: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:172.)
  return to_device(torch.f

vocoder.upsamples.0.0 
 torch.return_types.topk(
values=tensor([0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806]),
indices=tensor([1214,   10, 1213, 1308,   12, 1299,    9, 1312,   11, 1001, 1309,  380,
        1310, 1215, 1311,  298, 1313, 1003,  381,  360])) 



vocoder.upsamples.1.0 
 torch.return_types.topk(
values=tensor([0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806]),
indices=tensor([1817,   82, 1699,  252,  197, 1820, 1265, 1830,  983, 1227, 1823,  377,
        1825, 1818, 1828, 1695, 1833, 1261,  912,   77])) 



vocoder.upsamples.2.0 
 torch.return_types.topk(
values=tensor([0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806]),
indices=tensor([5122, 5106, 5118, 5127, 2559, 5126, 5115, 5133, 5089, 5111, 5128, 5108,
        5129, 5125, 5130, 5117, 5135, 5113, 5109, 5107])) 



vocoder.upsamples.3.0 
 torch.return_types.topk(
values=tensor([0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806]),
indices=tensor([ 6738,  5718,  5998,  7874,  4404,  7641,  5822,  9669,  4477,  4609,
         7877,  4346,  9109,  7115,  9442,  5843, 17926,  5712,  4401,  4212])) 



...

vocoder.blocks.11.convs2.0.0 
 torch.return_types.topk(
values=tensor([0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806]),
indices=tensor([60057, 41851, 60028, 60135, 41405, 60129, 41853, 60148, 41430, 41622,
        60139, 13204, 60144, 60090, 60147, 41855, 60152, 41845, 41404, 13194])) 



vocoder.blocks.11.convs2.1.0 
 torch.return_types.topk(
values=tensor([0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806]),
indices=tensor([28044, 27254, 28043, 41494, 26724, 38970, 27532, 43521, 26725, 27252,
        41495, 13513, 41496, 32675, 41497, 28042, 44276, 27253, 26179, 13216])) 



vocoder.blocks.11.convs2.2.0 
 torch.return_types.topk(
values=tensor([0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806]),
indices=tensor([ 6422,  5807,  5811, 11735,   292, 11732,   353, 19974,  5803,  5805,
        12725,   583, 17120, 10705, 17126,  5808, 20540,  5806,  5800,   291])) 



vocoder.output_conv.0 
 torch.return_types.topk(
values=tensor([0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806]),
indices=tensor([1162, 1126, 1161, 1204, 1019, 1183, 1139, 1288, 1064, 1070, 1221,  670,
        1266,  502, 1272, 1160, 1289, 1071,  952,  539])) 



vocoder.output_conv.2 
 torch.return_types.topk(
values=tensor([0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806,
        0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0806, 0.0720,
        0.0720, 0.0720]),
indices=tensor([27243, 12409, 20149, 28033, 11468, 27245, 12410, 12411, 11838, 11839,
        28034,  6791, 28035, 27244, 12408,  6792,  6790,  5817,  5820,  5816])) 

